In [1]:
ip_port = "132.72.66.220:9991"


In [2]:
import requests

# Define the URL and payload
url = f"http://{ip_port}/api/pull"
payload = {
    "model": "qwen3:32b"
}

# Make a POST request
response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Response:", response)

Response: <Response [200]>


In [3]:
url = f"http://{ip_port}/api/generate"

In [ ]:
import json

def extract_full_text(response_text):
    """
    This function extracts and concatenates text from the 'response' field in each JSON line of the response.
    
    Parameters:
    response_text (str): The raw response text, with each line being a JSON string.

    Returns:
    str: The concatenated text from all 'response' fields.
    """
    full_text = ""  # Variable to store the complete text
    
    # Iterate over each line in the response
    for line in response_text.text.splitlines():
        try:
            data = json.loads(line)  # Parse each JSON line
            if "response" in data:
                full_text += data["response"]  # Append the text
        except json.JSONDecodeError:
            print("Unable to parse line:", line)
    
    return full_text

In [5]:
import requests

def classify(Question):
    prompt = f"""
       
        أجب عن السؤال التالي في خمس إلى ست جمل.
        أجب على السؤال باللغة العربية.

        {Question}
       
"""
    
    payload = {
        "model": "qwen3:32b",
        "prompt": prompt,
        "temperature": 0,  
        "top_p": 0.1
    }

#     print(prompt)
    # Make a POST request
    response = requests.post(url, json=payload)

    # Validate the response
    if response.status_code == 200:
#         print(response.text)
        decision = extract_full_text(response)  # Extract the decision from the response
        return decision  # Return the decision
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [6]:
# !pip install pandas openpyxl

In [7]:
import pandas as pd

# Replace with your actual file path and sheet name
file_path = 'questions.xlsx'
sheet_name = 'arabic'  # or the exact name of the sheet

# Read the specific sheet
df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')

# Print the first few rows
print(df.head())


   id                         arabic
0   1               كيف تتشكل السحب؟
1   2        كيف انقرضت الديناصورات؟
2   3               ما هو الديناصور؟
3   4  لماذا تختلف ألوان عيون البشر؟
4   5     ما الفرق بين الذئب والكلب؟


In [8]:
df.iloc[0].arabic

'كيف تتشكل السحب؟'

In [9]:
classify(df.iloc[0].arabic)

"<think>\nOkay, the user is asking how clouds form, and they want the answer in 5 to 6 sentences in Arabic. Let me start by recalling the basic process. Clouds form when water vapor in the air condenses into droplets. I need to explain the steps clearly.\n\nFirst, the sun heats the Earth's surface, causing water to evaporate. Then, the warm, moist air rises. As it ascends, it cools down. When the air cools to the dew point, the water vapor condenses around tiny particles like dust or salt. These condensed droplets gather to form clouds. Maybe I should mention the role of condensation nuclei here. Also, different types of clouds can form depending on altitude and weather conditions. I should keep it concise, making sure each sentence covers a key step without getting too technical. Let me check if I have all the necessary points and structure it smoothly in Arabic.\n</think>\n\nتشكل السحب نتيجة تكثُّف بخار الماء في الغلاف الجوي. تبدأ العملية عندما تسخن أشعة الشمس سطح الأرض، مما يزيد من 

In [10]:
# d= pd.read_csv('deepseek/tweets_FDA.csv')
df.shape[0]

40

In [13]:
import os
import pandas as pd
from tqdm import tqdm

def get_last_processed_user(output_file):
    try:
        # Read the last line of the file to get the last processed user
        with open(output_file, 'r', encoding='utf-8-sig') as file:
            last_line = file.readlines()[-1]
            return last_line.split(',')[0]  # Assuming 'screen_name' is the first column
    except:
        # If there's an error (e.g., file is empty or doesn't exist), return None
        return None

def add_decision_column_and_save(df, output_file):
    # Check if the file already exists
    file_exists = os.path.exists(output_file)
    last_processed_user = get_last_processed_user(output_file) if file_exists else None
    start_processing = False if last_processed_user else True
    i = 0
    # Open the CSV file for appending
    with open(output_file, 'a', encoding='utf-8-sig', newline='') as file:
        # Iterate over rows with tqdm for progress tracking
        for i in tqdm(range(len(df)), desc="Processing rows"):
            content = df.iloc[i]['arabic']
            # if i < number:
            #     i+=1
            #     continue

            # # Start processing if it's the first run or after the last processed user
            # if start_processing or screen_name == last_processed_user:
            #     start_processing = True

            # name = df.iloc[i]['name']
            
            # Call the classify_user function to determine if it's an individual or organization
            decision = classify(content)
            
            # Prepare the current row with the decision and only the relevant columns
            current_row = {
                'text': content,
#                 'label':df.iloc[i]['label'],
                'classify': decision,
#                 'number': decision[-5:]
            }
            
            # Convert the row to a DataFrame
            row_df = pd.DataFrame([current_row])
            
            # Append the row to the CSV file
            row_df.to_csv(file, mode='a', index=False, header=not file_exists)
            file_exists = True  # Ensure the header is written only once

# File to save the updated DataFrame
output_file = "qwen3_arabic_3.csv"

# Assuming 'df' is your DataFrame
add_decision_column_and_save(df, output_file)

# Verify the result (optional)
print("Processing completed!")


Processing rows: 100%|██████████| 40/40 [07:46<00:00, 11.66s/it]

Processing completed!
